### X-ray Analysis
1. `If space between wings is a cavity, how much work is done by the rising X-ray bubble`



In [1]:
path_to_beads = '/Users/osaseomoruyi/Dropbox (Harvard University)/BeadsMultiwavelength/'

In [2]:
#system
from __future__ import division
import sys

#numpy
import numpy as np

import h5py
from types import SimpleNamespace

#matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib import cm
from matplotlib.colors import ListedColormap
from palettable.colorbrewer.qualitative import Set1_4 as brewermap
%matplotlib inline


#astropy
from astropy.wcs import WCS
import astropy.units as u
import astropy.constants as const
from astropy.cosmology import LambdaCDM
from astropy.coordinates import Angle
from astropy.io import ascii

#pandas
import pandas as pd

#scipy
import scipy

#seaborn
import seaborn as sns 

import importlib


In [3]:
#import my own functions: you can see them in full in the utils folder
util_path = path_to_beads + 'Notebooks/Beads20/utils/'
sys.path.append(util_path)

import plotting_functions as pf
import science_functions as sf

In [4]:
#uncomment if need to reload module
importlib.reload(pf)

<module 'plotting_functions' from '/Users/osaseomoruyi/Dropbox (Harvard University)/BeadsMultiwavelength/Notebooks/Beads20/utils/plotting_functions.py'>

In [5]:
#paths
fig_path = path_to_beads + 'Figures/paper/'
chandra_data_directory = '/Users/osaseomoruyi/chandra/alternate_tmap/alternate_spectral_maps_working_dir/merged/'
other_chandra_path = '/Users/osaseomoruyi/chandra/merged/'

In [6]:
#About my galaxy cluster
name = 'SDSS J1531+3414'
ra = 232.7936938
dec = 34.2404172
radius = Angle(2.5, u.arcsec)

zh  = 0.335 #z selected from Hennawi (2008)
cz = zh * const.c.to('km/s') # The stellar systemic velocity that we'll subtract off of our velocity maps. I always use Astropy Units - read up on them if you're not famililar!

cosmo = LambdaCDM(H0=71, Om0=0.27, Ode0=0.73)
da = cosmo.angular_diameter_distance(zh)
dl = cosmo.luminosity_distance(zh)

angular_diameter_distance = cosmo.angular_diameter_distance(z=zh) # in Mpc
kpc_per_arcsec = cosmo.kpc_proper_per_arcmin(z=zh).to(u.kpc / u.arcsec)

In [7]:
#plot style
pf.styleplots()

### Load Data

In [8]:
chandra_wavelet_file = ''.join((path_to_beads, 'Analysis/chandraBeads/wavelet_fit/beads_lynx_img.fits'))
chandra_raw_file = ''.join((path_to_beads, 'Analysis/chandraBeads/merged/beads_xray_bin2_broad_flux.img'))

In [10]:
#plot HST on top
hst_hdr, hst_wcs, hst_hdu = sf.load_HST_data(path_to_beads)
ysc_coords = sf.ysc_load(path_to_beads)
hst_cont_color, ysc_color = 'k', 'cyan'

### If space between wings is a cavity, was the heat released enough to mitgate ICM cooling?

In [12]:
specfit_savepath = path_to_beads + 'Analysis/chandraBeads/ciao_spectra_results/'
cel_flux = np.load(specfit_savepath + 'cf_60.npy')
cel_flux_err = np.load(specfit_savepath + 'cf_err_60.npy')

flux = np.load(specfit_savepath + 'flux_60.npy')
flux_min_err = np.load(specfit_savepath + 'flux_min_60.npy')
flux_max_err = np.load(specfit_savepath + 'flux_max_60.npy')

In [13]:
#KT and norm
kTs = np.load(specfit_savepath + 'kTs_60.npy') * u.keV
kT_errs = np.load(specfit_savepath + 'kT_err_60.npy') * u.keV
norms = np.load(specfit_savepath + 'norms_60.npy')
norm_errs = np.load(specfit_savepath + 'norm_errs_60.npy')


radius_df = pd.DataFrame(np.load(specfit_savepath + 'radius_60.npy'), columns=['radius', 
                            'radius_err', 'r_inner', 'r_outer'])

r_inner_in_kpc = radius_df['r_inner'].to_numpy() * u.kpc 
r_outer_in_kpc = radius_df['r_outer'].to_numpy() * u.kpc 
radius_to_plot = radius_df['radius'].to_numpy() * u.kpc 
radius_err_to_plot = radius_df['radius_err'].to_numpy() * u.kpc 

volume = (4/3) * scipy.pi * (r_outer_in_kpc**3 - r_inner_in_kpc**3)

#electron density
electron_density = np.sqrt((norms * u.cm**-5 * 4 * np.pi * 
                            np.power(angular_diameter_distance.to(u.cm) * (1 + zh), 2)) 
                            / (1.e-14 * 0.82 * volume.to(u.cm**3))) 
electron_density_err = electron_density / (2*norms) * norm_errs

#pressure
pressure = (2 * electron_density * kTs).to(u.dyne/u.cm**2)
pressure_err  = pressure * sf.dm_error_prop([electron_density, kTs], [electron_density_err, kT_errs])

#entropy
entropy  = kTs * np.power(electron_density, -2.0/3.0)
entropy_err = entropy * sf.dm_error_prop([kTs.value, np.power(electron_density, -2/3).value], 
                [kT_errs.value, sf.exp_error_prop(electron_density, electron_density_err, -2/3) ])

#cooling time
cooling_time = sf.tcool(electron_density, kTs)
cool_func_err = sf.dm_error_prop([kTs**-1.7, kTs**0.5], [sf.exp_error_prop(kTs, kT_errs, -1.7), 
                    sf.exp_error_prop(kTs, kT_errs, 0.5)]) * sf.coolingFunction(kTs)

cooling_time_err = cooling_time * sf.dm_error_prop([pressure.value, np.power(electron_density, 2).value, sf.coolingFunction(kTs).value],
                    [pressure_err.value, sf.exp_error_prop(electron_density, electron_density_err, 2).value, cool_func_err])

In [40]:
luminosity  = flux * u.erg / u.s / u.cm**2  * 4.0 * np.pi * np.power(cosmo.luminosity_distance(zh), 2) # in erg/s
luminosity_min_err  = luminosity * np.sqrt(((flux - flux_min_err)/flux)**2) 
luminosity_max_err  = luminosity * np.sqrt(((flux_max_err - flux)/flux)**2) 

In [42]:
print("Cooling Luminosity:")
print(luminosity.to(u.erg/u.s)[3], luminosity_min_err.to(u.erg/u.s)[3], luminosity_max_err.to(u.erg/u.s)[3])

Cooling Luminosity:
2.1534477158491653e+43 erg / s 1.4088857884116153e+42 erg / s 1.4104781519531465e+42 erg / s


In [43]:
def cavity_power(R, radius_cav, pressure_cav, sigma, C=0.75, gamma=5./3., kT=2.3, mu=0.62):
    """
    Calculate the cavity power of an ellipsoidal cavity in the ICM.

    Parameters
    ----------
    R : float
        Projected distance from the center of the cavity to the BCG center [kpc].
    radius_cav : float
        Projected radius of the cavity [kpc].
    pressure_cav : float
        pressure of outer edges of bubbble
    sigma : float
        Stellar velocity dispersion of the BCG [km/s].
    C : float, optional
        Drag coefficient, default to 0.75.

    Returns
    -------
    P_cav : float
        Cavity power [erg/s].
    """
    g = 2 * sigma**2 / R  # Gravitational acceleration [cm/s^2]
    S =   4 * np.pi * radius_cav**2 #cross sectional area of cavity
    V = 4 * np.pi * radius_cav**3 #volume cavity
 
    v_sound = np.sqrt(2*g*V/(C*S)) #np.sqrt(gamma * (kT*u.keV)/(mu * const.m_p))
    print(v_sound.to(u.km/u.s))
    t_cav = R /v_sound   # Cavity age [s]
    print('cav age: {}'.format(t_cav.to(u.year)/1e7))
    E_cav = 4 * pressure_cav * V  # Cavity energy [erg]
    P_cav = E_cav / t_cav  # Cavity power [erg/s]
    
    return P_cav.to(u.erg/u.s)


In [44]:
radius_cavity = 2.868 * u.arcsec * kpc_per_arcsec #kpc
distance_to_cavity = 4.338 * u.arcsec * kpc_per_arcsec #20.6 kpc
pressure_cavity = pressure[2] # units (u.dyne/u.cm**2)
pressure_cavity_err = pressure_err[2] # units (u.dyne/u.cm**2)
sigma = 444 * u.km/u.s #sdss
sigma_err = 100 * u.km/u.s

power_cavity = cavity_power(distance_to_cavity, radius_cavity, pressure_cavity, sigma)
power_cavity_err = power_cavity * sf.dm_error_prop([pressure_cavity, sigma], [pressure_cavity_err, sigma_err])

#cavity_power(distance_to_cavity, radius_cavity, pressure_cavity_err, sigma_err)
print("Cavity Power:")
print(power_cavity, power_cavity_err)

833.7332727704235 km / s
cav age: 2.4252893758544802 yr
Cavity Power:
1.5518633438528683e+45 erg / s 3.502304542762832e+44 erg / s
